In [1]:
%pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 13.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requi

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from gensim.models.ldamodel import LdaModel

# Load the LDA model
loaded_lda_model = LdaModel.load('/content/drive/MyDrive/Colab Notebooks/alex/models/lda_30_topics/lda_model_30_topics.gensim')

# Now you can access information from the loaded model.
# For example, you can print the topics:
print("Topics:")
for topic_id, topic in loaded_lda_model.print_topics():
    print(f"Topic {topic_id}: {topic}")

# You can also access the dictionary (id2word)
print("\nDictionary:")
print(loaded_lda_model.id2word)

# And the number of topics
print("\nNumber of topics:")
print(loaded_lda_model.num_topics)

Topics:
Topic 5: 0.020*"ncaa" + 0.008*"university" + 0.007*"investigation" + 0.007*"statement" + 0.007*"court" + 0.006*"told" + 0.006*"settlement" + 0.006*"school" + 0.005*"players" + 0.005*"federal"
Topic 13: 0.029*"sports" + 0.027*"espn" + 0.026*"tv" + 0.023*"network" + 0.020*"watch" + 0.020*"usa" + 0.017*"cbs" + 0.015*"live" + 0.013*"media" + 0.013*"games"
Topic 25: 0.031*"tournament" + 0.027*"seed" + 0.025*"final" + 0.024*"ncaa_tournament" + 0.021*"team" + 0.019*"round" + 0.018*"teams" + 0.013*"win" + 0.012*"march_madness" + 0.012*"south"
Topic 28: 0.106*"gophers" + 0.089*"minnesota" + 0.036*"ten" + 0.026*"johnson" + 0.021*"staff" + 0.012*"season" + 0.011*"lynx" + 0.010*"team" + 0.010*"vikings" + 0.009*"startribune.com"
Topic 29: 0.122*"indiana" + 0.065*"gonzaga" + 0.061*"miller" + 0.052*"calipari" + 0.030*"bulldogs" + 0.029*"memphis" + 0.028*"knight" + 0.025*"butler" + 0.024*"hoosiers" + 0.022*"sampson"
Topic 7: 0.017*"golf" + 0.013*"united" + 0.010*"won" + 0.010*"round" + 0.009*"

In [3]:
from gensim import corpora

# Load the corpus from the .mm file
corpus = corpora.MmCorpus('/content/drive/MyDrive/Colab Notebooks/alex/models/dictionary_and_corpus/corpus.mm')

print("Corpus loaded successfully.")
# You can optionally print the corpus to inspect its structure
# print(list(corpus))

Corpus loaded successfully.


In [4]:
import json
import pandas as pd

# Assuming your JSON file contains a list of lists, where each inner list is a tokenized document
# Replace '/content/drive/MyDrive/path/to/your/texts.json' with the actual path to your JSON file
texts_file_path = '/content/drive/MyDrive/Colab Notebooks/alex/models/dictionary_and_corpus/texts.json'

with open(texts_file_path, 'r') as f:
    texts = json.load(f)

print(f"Loaded {len(texts)} documents from {texts_file_path}")

Loaded 149649 documents from /content/drive/MyDrive/Colab Notebooks/alex/models/dictionary_and_corpus/texts.json


In [ ]:
# import pandas as pd

# # Load the tokenized data from the Excel file
# try:
#     tokenized_data_df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/alex/updated_ tokenized_newspaper_data.xlsx')
#     print("Excel file loaded successfully.")
#     # Assuming the original text is in a column named 'text' or similar.
#     # Please adjust the column name if it's different in your Excel file.
#     if 'Main_text_tokenized' in tokenized_data_df.columns:
#         texts = tokenized_data_df['Main_text_tokenized'].tolist()
#         print(f"Extracted {len(texts)} documents.")
#     else:
#         texts = None
#         print("Could not find a 'text' column in the Excel file. Please make sure the column containing original texts is named 'text' or update the code.")

# except FileNotFoundError:
#     print("Error: The file was not found at the specified path.")
# except Exception as e:
#     print(f"An error occurred while reading the Excel file: {e}")

Excel file loaded successfully.
Extracted 149703 documents.


In [5]:
#TFinding the dominant topic in each sentence

def format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts): # Updated to use loaded_lda_model and corpus, and texts=None
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = pd.concat([sent_topics_df, pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]).to_frame().T], ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

     # Add original text to the end of the output
    if texts is not None: # Added check if texts is not None
        contents = pd.Series(texts)
        sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


# Assuming texts is available as data_words or you provide another list/Series of texts
# df_topic_sents_keywords = format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts) # Updated to use loaded_lda_model and corpus, commented out as data_words is undefined
# To run this cell, define `data_words` with your text data before calling the function.
# For example:
# data_words = ["your document text 1", "your document text 2", ...]
# Then uncomment the line below:
df_topic_sents_keywords = format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts)

# Format
if 'df_topic_sents_keywords' in locals(): # Added check if df_topic_sents_keywords is defined
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
    # Show
    display(df_dominant_topic.head(20)) # Use display for better formatting

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,4,0.4485,"people, time, day, life, told, fans, family, b...","[coaching, living, bonnie, henrickson, phone, ..."
1,1,9,0.3465,"coach, program, coaches, season, head_coach, c...","[sidebar, ruston, la, dining, louisiana, cajun..."
2,2,24,0.2823,"half, lead, minutes, shot, left, scored, play,...","[garden, missed, lonely, nights, season, debut..."
3,3,4,0.2101,"people, time, day, life, told, fans, family, b...","[ten, ago, chuck, daly, reclining, alongside, ..."
4,4,5,0.2208,"ncaa, university, investigation, statement, co...","[robinson, talk, rangers, stanley, cup, finals..."
5,5,19,0.3063,"season, guard, rebounds, player, team, forward...","[conference, pacific, coach_mike, gillespie, s..."
6,6,16,0.4464,"hockey, goals, nhl, goal, boston, games, ice, ...","[rangers, signed, center, michael, nylander, m..."
7,7,1,0.2174,"nba, league, draft, basketball, player, pick, ...","[gavitt, elected, hall, modest, player, dartmo..."
8,8,24,0.3061,"half, lead, minutes, shot, left, scored, play,...","[mythical, element, senior_guard, seimone, aug..."
9,9,4,0.3418,"people, time, day, life, told, fans, family, b...","[ncaa_women, jim, izard, diana, vines, remembe..."


In [6]:
# Display the first few rows of the dataframe to see the structure
display(df_dominant_topic.head())

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,4,0.4485,"people, time, day, life, told, fans, family, b...","[coaching, living, bonnie, henrickson, phone, ..."
1,1,9,0.3465,"coach, program, coaches, season, head_coach, c...","[sidebar, ruston, la, dining, louisiana, cajun..."
2,2,24,0.2823,"half, lead, minutes, shot, left, scored, play,...","[garden, missed, lonely, nights, season, debut..."
3,3,4,0.2101,"people, time, day, life, told, fans, family, b...","[ten, ago, chuck, daly, reclining, alongside, ..."
4,4,5,0.2208,"ncaa, university, investigation, statement, co...","[robinson, talk, rangers, stanley, cup, finals..."


In [7]:
# Sort documents within each topic by percentage contribution in descending order
sorted_documents_by_topic = df_dominant_topic.sort_values(by=['Dominant_Topic', 'Topic_Perc_Contrib'], ascending=[True, False])

# Display the sorted document indexes for each topic
print("Document Indexes for Each Topic (Sorted by Percentage Contribution):")
for topic_num in sorted_documents_by_topic['Dominant_Topic'].unique():
    print(f"\nTopic {topic_num}:")
    # Get documents for the current topic and select only the 'Document_No' column
    topic_documents = sorted_documents_by_topic[sorted_documents_by_topic['Dominant_Topic'] == topic_num]['Document_No']
    # Display the document indexes (first 20 as an example)
    print(topic_documents.tolist()[:20])

Document Indexes for Each Topic (Sorted by Percentage Contribution):

Topic 0:
[7767, 143375, 145791, 60318, 76010, 25102, 139776, 122114, 102530, 141855, 108040, 131394, 141745, 122113, 119658, 28358, 122607, 102052, 143838, 131384]

Topic 1:
[29134, 60308, 35678, 117624, 50512, 13271, 24210, 46827, 120014, 120015, 1242, 14715, 50511, 50513, 80593, 145723, 4843, 101871, 85984, 120031]

Topic 2:
[10627, 37592, 72246, 138322, 98680, 70660, 65224, 60670, 50073, 12518, 70093, 138229, 55312, 80907, 37866, 69697, 93669, 14954, 78306, 93149]

Topic 3:
[83656, 131802, 94996, 94717, 100379, 19333, 11292, 94994, 112119, 101869, 43348, 8995, 53822, 17582, 43636, 139656, 143432, 8994, 91918, 52055]

Topic 4:
[126048, 30675, 35009, 40852, 88515, 43671, 37172, 71271, 9888, 31041, 143254, 107896, 62136, 39586, 76946, 1075, 54520, 27417, 1977, 145662]

Topic 5:
[31118, 115822, 25870, 127167, 127332, 77012, 28071, 67540, 100438, 124713, 110794, 56172, 24077, 56468, 61206, 121819, 33888, 14542, 49104, 

In [8]:
# Get the total number of documents for each dominant topic
topic_counts = df_dominant_topic['Dominant_Topic'].value_counts().sort_index()

print("Total number of documents for each topic:")
display(topic_counts)

Total number of documents for each topic:


,count
Dominant_Topic,
0,855
1,4797
2,8434
3,3215
4,14350
5,7298
6,303
7,8390
8,785


In [9]:
# Get the top 100 documents for each topic
top_100_documents_per_topic = pd.DataFrame()

for topic_num in sorted_documents_by_topic['Dominant_Topic'].unique():
    # Get the top 100 documents for the current topic
    top_100 = sorted_documents_by_topic[sorted_documents_by_topic['Dominant_Topic'] == topic_num].head(100)
    # Append to the results DataFrame
    top_100_documents_per_topic = pd.concat([top_100_documents_per_topic, top_100], ignore_index=True)

# Display the top 100 documents for each topic
print("Top 100 Documents for Each Topic (Sorted by Percentage Contribution):")

# Iterate through topics and display the top documents for each
for topic_num in top_100_documents_per_topic['Dominant_Topic'].unique():
    print(f"\n--- Topic {topic_num} ---")
    display(top_100_documents_per_topic[top_100_documents_per_topic['Dominant_Topic'] == topic_num])

Top 100 Documents for Each Topic (Sorted by Percentage Contribution):

--- Topic 0 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,7767,0,0.671,"women, olympic, sports, olympics, world, gold,...","[ncaa, team, champions, women, lacrosse, baseb..."
1,143375,0,0.581,"women, olympic, sports, olympics, world, gold,...","[consecutive, division, championships, sport, ..."
2,145791,0,0.5601,"women, olympic, sports, olympics, world, gold,...","[wrestling, paris, olympics, wrestling, olympi..."
3,60318,0,0.5265,"women, olympic, sports, olympics, world, gold,...","[mariel, zagunis, zagunis, performance, follow..."
4,76010,0,0.5196,"women, olympic, sports, olympics, world, gold,...","[teams, targeted, elimination, operating, expe..."
...,...,...,...,...,...
95,28356,0,0.3561,"women, olympic, sports, olympics, world, gold,...","[saturday, aug, swimming, competition, olympic..."
96,110292,0,0.356,"women, olympic, sports, olympics, world, gold,...","[olympic, gold, medalist, time_ncaa_champion, ..."
97,142232,0,0.3551,"women, olympic, sports, olympics, world, gold,...","[paris, lee, kiefer, smiled, people, witnessed..."
98,128521,0,0.3532,"women, olympic, sports, olympics, world, gold,...","[sarah, bacon, tokyo, week, straightforward, t..."



--- Topic 1 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
100,29134,1,0.7808,"nba, league, draft, basketball, player, pick, ...","[pro, basketball, free_agent, guard, doug, chr..."
101,60308,1,0.7563,"nba, league, draft, basketball, player, pick, ...","[timberwolves, entered, thursday, night, draft..."
102,35678,1,0.7359,"nba, league, draft, basketball, player, pick, ...","[bucks, suffered, disappointing, season, playo..."
103,117624,1,0.7222,"nba, league, draft, basketball, player, pick, ...","[wearing, lakers, hat, standing, front, lakers..."
104,50512,1,0.7196,"nba, league, draft, basketball, player, pick, ...","[york, gators, ncaa_champions, nba_draft, hist..."
...,...,...,...,...,...
195,145578,1,0.5804,"nba, league, draft, basketball, player, pick, ...","[college_basketball, biggest, stars, caitlin_c..."
196,9966,1,0.5801,"nba, league, draft, basketball, player, pick, ...","[denver, nuggets, telling, keeping, pick, draf..."
197,90720,1,0.5796,"nba, league, draft, basketball, player, pick, ...","[corey, brewer, played, games, spent, months, ..."
198,97154,1,0.5792,"nba, league, draft, basketball, player, pick, ...","[dwyane, wade, choice, chris, bosh, mulling, f..."



--- Topic 2 ---


...

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
2590,124676,27,0.8095,"trump, president, city, people, sports_betting...","[glendale, resort, style, development, anchore..."
2591,124060,27,0.8035,"trump, president, city, people, sports_betting...","[glendale, resort, style, development, anchore..."
2592,138700,27,0.7851,"trump, president, city, people, sports_betting...","[convention, center, named, top, convention, c..."
2593,120106,27,0.7811,"trump, president, city, people, sports_betting...","[hillsborough, county, set, records, tourism, ..."
2594,16543,27,0.7721,"trump, president, city, people, sports_betting...","[jay, leno, espn, remote, broadcast, locations..."
...,...,...,...,...,...
2685,16743,27,0.6554,"trump, president, city, people, sports_betting...","[st, patrick, day, eventful, holiday, vegas, c..."
2686,40197,27,0.6553,"trump, president, city, people, sports_betting...","[mike, kalil, review, journal, private, propos..."
2687,139386,27,0.6552,"trump, president, city, people, sports_betting...","[ben, graves, taylor, swift, biggest, weekend,..."
2688,70928,27,0.655,"trump, president, city, people, sports_betting...","[political, junkies, tuesday, equivalent, sele..."



--- Topic 28 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
2690,86231,28,0.6944,"gophers, minnesota, ten, johnson, staff, seaso...","[xcel, energy, center, play, host, west_region..."
2691,12728,28,0.5577,"gophers, minnesota, ten, johnson, staff, seaso...","[gophers, hockey, mercyhurst, p.m, friday, rou..."
2692,137456,28,0.5179,"gophers, minnesota, ten, johnson, staff, seaso...","[garrett, hendrickson, assistant_coach, past, ..."
2693,19537,28,0.5084,"gophers, minnesota, ten, johnson, staff, seaso...","[ncaa_champions, dated, recent, ncaa_hockey, c..."
2694,141171,28,0.5063,"gophers, minnesota, ten, johnson, staff, seaso...","[story, thursday, ncaa_wrestling, championship..."
...,...,...,...,...,...
2785,113016,28,0.3625,"gophers, minnesota, ten, johnson, staff, seaso...","[week, bemidji, st_cloud, p.m, saturday, teams..."
2786,27783,28,0.3624,"gophers, minnesota, ten, johnson, staff, seaso...","[staff, reports, gophers_basketball_team, bene..."
2787,88350,28,0.3624,"gophers, minnesota, ten, johnson, staff, seaso...","[north_dakota, p.m, friday, p.m, saturday, mar..."
2788,82152,28,0.3618,"gophers, minnesota, ten, johnson, staff, seaso...","[winona, minnesota_duluth, p.m, saturday, warr..."



--- Topic 29 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
2790,57037,29,0.8066,"indiana, gonzaga, miller, calipari, bulldogs, ...","[east_regional, south_regional, midwest_region..."
2791,83655,29,0.5044,"indiana, gonzaga, miller, calipari, bulldogs, ...","[ncaa, scores, east_regional, gonzaga, south_r..."
2792,60480,29,0.4961,"indiana, gonzaga, miller, calipari, bulldogs, ...","[round, scoreboard, east_regional, pitt, xavie..."
2793,50374,29,0.4439,"indiana, gonzaga, miller, calipari, bulldogs, ...","[photograph, antonio, graves, pitt, smothered,..."
2794,102740,29,0.4214,"indiana, gonzaga, miller, calipari, bulldogs, ...","[ncaa, scoreboard, gonzaga, n.c, illustration,..."
2795,46127,29,0.3574,"indiana, gonzaga, miller, calipari, bulldogs, ...","[thad, matta, left, xavier, ago, musketeer, fa..."
2796,24334,29,0.3262,"indiana, gonzaga, miller, calipari, bulldogs, ...","[xavier, regional_finals, time, time, refreshe..."
2797,47044,29,0.2941,"indiana, gonzaga, miller, calipari, bulldogs, ...","[coaching, wins, coaches, victories, spent, mi..."
2798,61458,29,0.2901,"indiana, gonzaga, miller, calipari, bulldogs, ...","[coach, calipari, agree, bobby, knight, colleg..."
2799,73950,29,0.2674,"indiana, gonzaga, miller, calipari, bulldogs, ...","[west_coast, champion, ncaa_tournament, held, ..."


In [10]:
# Define the path to save the Excel file in your Google Drive
excel_file_path = '/content/drive/MyDrive/Colab Notebooks/alex/top_100_articles/top_100_documents_per_topic_for_30_topics.xlsx'

# Save the DataFrame to an Excel file
try:
    top_100_documents_per_topic.to_excel(excel_file_path, index=False)
    print(f"Successfully saved the top 100 documents per topic to: {excel_file_path}")
except Exception as e:
    print(f"An error occurred while saving the Excel file: {e}")

Successfully saved the top 100 documents per topic to: /content/drive/MyDrive/Colab Notebooks/alex/top_100_articles/top_100_documents_per_topic_for_30_topics.xlsx


##### **© 2024–2025 MD Rafsun Sheikh**
##### **Licensed under the Apache License, Version 2.0.**